In [ ]:
import nltk
import scipy
import pandas as pd
import numpy as np

import csv
from nltk.corpus import wordnet as wn
from libs import *

In [ ]:
for i in range(0,353):
    v1 = word_sym.at[i,'Word 1']
    v2 = word_sym.at[i,'Word 2']
    max_wu=0
    max_sp=0
    max_lch=0
    for s1 in wn.synsets(v1):
        for s2 in wn.synsets(v2):
            actual_wu=wu_similarity(v1,v2)
            actual_sp=sp_similarity(v1,v2)
            actual_lch=lch_similarity(v1,v2)
            if actual_wu>max_wu:
                max_wu=actual_wu
            if actual_sp>max_sp:
                max_sp=actual_sp
            if actual_lch>max_lch:
                max_lch=actual_lch
    word_sym.iat[i,3]= max_wu
    word_sym.iat[i,4]= max_sp
    word_sym.iat[i,5]= max_lch
